In [1]:
#What is the problem??
#It dramatically depends on the kind of fantasy league you are playing
#(1)Trying to predict which player combinations will win you the most number of points in any given week?
#(2)Which players are going to be the best players next week/year/month?
#One thing that is interesting is that best players probably, but might not, add up to the best teams



In [2]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup


In [3]:
#http://danielfrg.com/blog/2013/04/01/nba-scraping-data/
url = 'http://espn.go.com/nba/teams'
r = requests.get(url)

soup = BeautifulSoup(r.text)
tables = soup.find_all('ul', class_='medium-logos')

teams = []
prefix_1 = []
prefix_2 = []
teams_urls = []
for table in tables:
    lis = table.find_all('li')
    for li in lis:
        info = li.h5.a
        teams.append(info.text)
        url = info['href']
        teams_urls.append(url)
        prefix_1.append(url.split('/')[-2])
        prefix_2.append(url.split('/')[-1])


dic = {'url': teams_urls, 'prefix_2': prefix_2, 'prefix_1': prefix_1}
teams = pd.DataFrame(dic, index=teams)
teams.index.name = 'team'
print(teams)
teams.to_csv("teams.csv")

                       prefix_1                prefix_2  \
team                                                      
Boston Celtics              bos          boston-celtics   
Brooklyn Nets               bkn           brooklyn-nets   
New York Knicks              ny         new-york-knicks   
Philadelphia 76ers          phi      philadelphia-76ers   
Toronto Raptors             tor         toronto-raptors   
Golden State Warriors        gs   golden-state-warriors   
LA Clippers                 lac             la-clippers   
Los Angeles Lakers          lal      los-angeles-lakers   
Phoenix Suns                phx            phoenix-suns   
Sacramento Kings            sac        sacramento-kings   
Chicago Bulls               chi           chicago-bulls   
Cleveland Cavaliers         cle     cleveland-cavaliers   
Detroit Pistons             det         detroit-pistons   
Indiana Pacers              ind          indiana-pacers   
Milwaukee Bucks             mil         milwaukee-bucks 

/Library/Python/2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 162 of the file /System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html.parser")

  markup_type=markup_type))


# Collect data from espn from 2003 to 2017

In [425]:
BASE_URL = 'http://www.espn.com/nba/team/stats/_/name/{0}/year/{1}'

column_headers = [u'PLAYER', u'GP', u'GS', u'MIN', u'PPG', u'OFFR', u'DEFR', u'RPG', u'APG', u'SPG', u'BPG', u'TPG', u'FPG', u'A/TO', u'PER']

def normalizeColumn(column):
    theMax = column.max()
    theMin = column.min()
    normalizedColumn = [ (record - theMin) / (theMax - theMin) for record in column]
    return normalizedColumn

for year in range(2003, 2018):
    player_data = []
    for index, row in teams.iterrows():
        r = requests.get(BASE_URL.format(row['prefix_1'], year))
        table = BeautifulSoup(r.text).table
        for row in table.find_all('tr')[2:][:-1]: # Remove header
            data = [td.getText() for td in row.findAll('td')]
            formattedData = [data[0].encode('utf-8')] + [float(x) for x in data[1:]] #formats the none-name fields to floats so we can do math
            player_data.append(formattedData)
    df = pd.DataFrame(player_data, columns=column_headers)
    for column in df.iloc[:,1:]:
        data = normalizeColumn(df[column])
        df[column + "_Normalized"] = data
    df['scores_Normalized'] = df.iloc[:,15:].apply(_sum, axis = 1)
    df.to_csv("player_stats_" + str(year))

9.01230482945
8.45591533179
5.24879644643
6.36921233351
4.61607905387
4.15147883637
5.99975355024
4.68627669487
4.18915032634
4.18208452345
2.26706365518
3.11540380266
2.55810508955
1.67789126593
0.905628742611
0.938404812496
1.18883603694
9.03696813667
8.21207510125
7.42698374644
6.1145983215
4.75423156612
4.61944699615
4.58335616494
4.69101610264
5.68096582171
3.25222749355
3.16818549293
2.08022281795
1.21431121364
2.276764528
0.392205414716
6.71874286517
7.00755231647
7.94469013753
6.44895586381
4.64883397578
5.81541845056
5.15883146431
5.64107821192
2.72268436438
3.52256668014
1.58703618595
2.41752164325
2.22121046952
8.71289219012
7.13497810702
7.80290915166
6.22980141168
6.18862512572
6.18835225145
4.56051173517
4.46424751426
4.56285176795
4.09161537916
2.98748521975
2.03508649534
2.56572787246
1.84318098471
1.54967400623
1.53433926422
6.64033038112
5.48053458205
6.84080413444
7.0498337845
7.03725164807
7.28421689649
4.07982421811
4.71099244192
4.947129505
3.00199359864
3.8445559

In [478]:
pd.read_csv('player_stats_2016').sort_values(by=['scores_Normalized'], ascending=False)

,Unnamed: 0,PLAYER,GP,GS,MIN,PPG,OFFR,DEFR,RPG,APG,...,DEFR_Normalized,RPG_Normalized,APG_Normalized,SPG_Normalized,BPG_Normalized,TPG_Normalized,FPG_Normalized,A/TO_Normalized,PER_Normalized,scores_Normalized
479,479,"Russell Westbrook, PG",80.0,80.0,34.4,23.5,1.8,6.0,7.8,10.4,...,0.582524,0.527027,0.888889,0.953271,0.067935,0.934783,0.416667,0.342857,0.640653,9.267327
155,155,"DeMarcus Cousins, PF",65.0,65.0,34.6,26.9,2.4,9.1,11.5,3.3,...,0.883495,0.777027,0.282051,0.724299,0.385870,0.826087,0.600000,0.128571,0.569873,8.962191
206,206,"Andre Drummond, C",81.0,81.0,32.9,16.2,4.9,9.9,14.8,0.8,...,0.961165,1.000000,0.068376,0.686916,0.375000,0.413043,0.500000,0.057143,0.526316,8.878585
270,270,"James Harden, SG",82.0,82.0,38.1,29.0,0.8,5.3,6.1,7.5,...,0.514563,0.412162,0.641026,0.794393,0.168478,1.000000,0.466667,0.228571,0.600726,8.857718
85,85,"Draymond Green, PF",81.0,81.0,34.7,14.0,1.7,7.8,9.5,7.4,...,0.757282,0.641892,0.632479,0.686916,0.380435,0.695652,0.500000,0.328571,0.490018,8.721837
83,83,"Stephen Curry, PG",79.0,79.0,34.2,30.1,0.9,4.6,5.4,6.7,...,0.446602,0.364865,0.572650,1.000000,0.051630,0.717391,0.333333,0.285714,0.713249,8.404309
354,354,"Paul Millsap, PF",81.0,81.0,32.7,17.1,2.4,6.6,9.0,3.3,...,0.640777,0.608108,0.282051,0.845794,0.467391,0.521739,0.483333,0.200000,0.526316,8.380930
424,424,"John Wall, PG",77.0,77.0,36.2,19.9,0.5,4.4,4.9,10.2,...,0.427184,0.331081,0.871795,0.878505,0.209239,0.891304,0.350000,0.357143,0.499093,8.313654
186,186,"LeBron James, SF",76.0,76.0,35.6,25.3,1.5,6.0,7.4,6.8,...,0.582524,0.500000,0.581197,0.640187,0.173913,0.717391,0.316667,0.300000,0.640653,8.295079
158,158,"Rajon Rondo, PG",72.0,72.0,35.2,11.9,1.1,5.0,6.0,11.7,...,0.485437,0.405405,1.000000,0.915888,0.038043,0.847826,0.400000,0.428571,0.446461,8.175396


# Train model and calculate mean absolute error on data from 2005 to 2016

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

player_stats_by_year = []
for year in range(2005, 2017):
    if(year != 2016):
        thisYear = pd.read_csv('player_stats_'+str(year))
        nextYear = pd.read_csv('player_stats_'+str(year+1))
        scores_next_year = nextYear[['PLAYER', 'scores_Normalized']]
        renamed_scores = scores_next_year.rename(index=str, columns={"scores_Normalized": "y"})
        merged = pd.merge(thisYear, renamed_scores, on='PLAYER')
        player_stats_by_year.append(merged)

all_player_data = pd.concat(player_stats_by_year)
lr = LinearRegression(n_jobs=-1)
X = all_player_data.iloc[:,2:16]
y = all_player_data.iloc[:,31]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
lr.fit(X_train,y_train)
y_pred = [lr.predict([row]) for label,row in X_test.iterrows()]
mean_absolute_error(y_test, y_pred)

# Test model on 2017 to predict for 2018

In [479]:
all_data = pd.read_csv('player_stats_2017')
X = pd.read_csv('player_stats_2017').iloc[:,2:16]
players = all_data.PLAYER
y_pred = [lr.predict([row])[0] for label,row in X.iterrows()]
pred_2017 = pd.concat([players,pd.Series(y_pred, name="score")], axis=1)
pred_2017.sort_values(by=['score'], ascending=False)

,PLAYER,score
426,"Russell Westbrook, PG",10.510057
249,"James Harden, SG",9.991176
280,"Anthony Davis, PF",9.501400
139,"DeMarcus Cousins, PF",9.441470
216,"Giannis Antetokounmpo, SF",9.104813
170,"LeBron James, SF",8.709306
411,"Karl-Anthony Towns, C",8.593087
379,"John Wall, PG",8.562143
188,"Andre Drummond, C",8.450731
78,"Kevin Durant, SF",8.267729
